In [ ]:

import os, cv2, numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

input_dir  = "path"
output_dir = "path"
os.makedirs(output_dir, exist_ok=True)

# Threshold & morphology settings
THRESH_VAL = 200
DILATE_KERNEL = (5,5)
DILATE_ITER  = 2

# Inpainting settings
INPAINT_RADIUS = 3
INPAINT_METHOD = cv2.INPAINT_TELEA


DO_CLAHE = True
DO_RESIZE = True
RESIZE_TO = (3000,3000)
CLAHE_CLIP = 2.0
CLAHE_TILE = (8,8)

def is_image_file(name):
    return name.lower().endswith((".png",".jpg",".jpeg",".bmp",".tif",".tiff"))

def remove_text(img):
    """Detect bright text and remove with inpainting."""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    _, thresh = cv2.threshold(gray, THRESH_VAL, 255, cv2.THRESH_BINARY)

    kernel = np.ones(DILATE_KERNEL, np.uint8)
    mask = cv2.dilate(thresh, kernel, iterations=DILATE_ITER)

    cleaned = cv2.inpaint(img, mask, INPAINT_RADIUS, INPAINT_METHOD)
    return cleaned, mask

def postprocess(img):
    out = img

    if DO_CLAHE:
        g = cv2.cvtColor(out, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=CLAHE_CLIP, tileGridSize=CLAHE_TILE)
        g = clahe.apply(g)
        out = cv2.cvtColor(g, cv2.COLOR_GRAY2BGR)

    if DO_RESIZE:
        out = cv2.resize(out, RESIZE_TO, interpolation=cv2.INTER_AREA)
    return out


def preview_one(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    cleaned, mask = remove_text(img)
    final = postprocess(cleaned)

    plt.figure(figsize=(16,6))
    plt.subplot(1,3,1); plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)); plt.title("Original"); plt.axis("off")
    plt.subplot(1,3,2); plt.imshow(mask, cmap="gray"); plt.title("Detected Text Mask"); plt.axis("off")
    plt.subplot(1,3,3); plt.imshow(cv2.cvtColor(final, cv2.COLOR_BGR2RGB)); plt.title("Cleaned (No Text)"); plt.axis("off")
    plt.show()



def process_all():
    files = [f for f in os.listdir(input_dir) if is_image_file(f)]
    print(f"Found {len(files)} images.")
    for f in tqdm(files, desc="Cleaning"):
        in_path = os.path.join(input_dir, f)
        img = cv2.imread(in_path, cv2.IMREAD_COLOR)
        if img is None: continue
        cleaned, _ = remove_text(img)
        final = postprocess(cleaned)
        out_path = os.path.join(output_dir, os.path.splitext(f)[0] + ".jpg")
        cv2.imwrite(out_path, final, [cv2.IMWRITE_JPEG_QUALITY, 95])
    print("✅ Done! Clean images saved to:", output_dir)

# Run on all images (uncomment):
process_all()
